In [1]:
import pandas as pd
import io
#Dependencies


from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain


#123

df=pd.read_excel('Example for data base.xlsx')
df

,Category,Removed,Removed 1,חותמת זמן,First name,Last (family) name,Platform,"Language of the post: if none of these options fit, please choose ""Other"". Then remember to complete the next question.",Content Link,Other language? Please specify below.,Reported?,Please upload a screenshot of the content,A short summary of the content- write briefly in your own words,Email address,Country where content published,Age of user
0,Y,X,X,2021-07-01 20:39:51,Ronit,Litmanowicz,Instagram,Spanish,https://www.instagram.com/p/B1-E4_RBJqO/?utm_s...,NaN,Yes,https://drive.google.com/open?id=1mNM7Rz3RUwVa...,#Genocidal Israel\n,ronitlitma@gmail.com,NaN,NaN
1,W,X,X,2021-07-01 20:42:35,Ronit,Litmanowicz,Instagram,Spanish,https://www.instagram.com/p/BsD1HF-hhKg/?utm_s...,NaN,Yes,https://drive.google.com/open?id=1aTADJkaDrb66...,#Genocidal Israel\n a picture of Binyamin Neta...,ronitlitma@gmail.com,NaN,NaN
2,U,X,X,2021-08-01 02:43:41,Daria,Kazakova,Twitter,English,https://twitter.com/BegomSahanara/status/12848...,NaN,Yes,https://drive.google.com/open?id=1tLBfvLYcLonl...,"denying Israel's right for a country, encourag...",daria140795@gmail.com,NaN,NaN
3,Y,X,X,2021-08-01 02:50:24,Daria,Kazakova,Twitter,English,https://twitter.com/MOHD_RAEES786/status/12849...,NaN,Yes,https://drive.google.com/open?id=1cSe4RpgBe_oN...,encourage to delete Israel from the map denyin...,daria140795@gmail.com,NaN,NaN
4,Y,X,X,2021-08-01 03:12:23,Daria,Kazakova,Twitter,English,https://twitter.com/LailaPalestini1/status/134...,NaN,Yes,https://drive.google.com/open?id=12SOC5at301hX...,lies about Israel for hate speech,daria140795@gmail.com,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,NaN,X,X,16/02/2021 12:19:04,מאור,קרופרו,Facebook,English,https://www.facebook.com/lilly.torre,ערבית,Yes,https://drive.google.com/open?id=1fEeTCK9ewhsJ...,אישה שמעלה סרטונים נגד מדינת ישראל,maur.krupru@gmail.com,European Union,NaN
297,NaN,X,X,16/02/2021 12:28:20,מאור,קרופרו,Facebook,English,https://www.facebook.com/amer.mansour.319,ערבית,Yes,https://drive.google.com/open?id=1kRLhxSyaPi4k...,פיסבוק של אדם ששונא ישראל ומעלה לשם תמונות אנט...,maur.krupru@gmail.com,European Union,NaN
298,Y,X,X,16/02/2021 12:35:55,ayana,bibiyan,YouTube,Russian,https://www.youtube.com/watch?v=BWIa5PP7Nqc&t=...,NaN,Yes,https://drive.google.com/open?id=1uJpU7wfRS5h2...,A video against Zionist Jews,ayanabibiyan@gmail.com,Unknown,NaN
299,Y,X,X,16/02/2021 18:42:42,Ronit,Litmanowicz,YouTube,Spanish,https://www.youtube.com/watch?v=yD5etBsBKkc&ab...,NaN,Yes,https://drive.google.com/open?id=1NsRJ7Hau_fx4...,"""Zionism is the most monstrous face of humanit...",ronitlitma@gmail.com,"Latin America (For example: Brazil, Argentina.,,)",NaN


In [ ]:
import pandas as pd

comments_fd = pd.read_csv('comments.csv')

print(comments_fd)

grouped_by_platform = df.groupby('Platform')

for socialmedia in grouped_by_platform:
    print(socialmedia)


import io



NameError: name 'generate_masks' is not defined

In [5]:

Instagram = df["Platform"]=="Instagram"
Twitter = df["Platform"]=="Twitter"
Facebook = df["Platform"]=="Facebook"
TikTok = df["Platform"]=="Tik Tok"
Youtube = df["Platform"]=="Youtube"

df_ig = df[Instagram]


In [14]:
list = ["Anti-Zionist",
"Antisemitic",
"Non-Antisemitic"]

df_ig = df_ig.loc[df_ig['Category'] .isin(list)]
df_category=df_ig['Category']).value_counts()

df_category.plot(kind='bar', stacked=True,mark_right = True )
#איפה שכתובה הרשת נתון לשינוי


SyntaxError: unmatched ')' (<ipython-input-14-62162b15d2f2>, line 6)

In [13]:
list = ["Anti-Zionist",
"Antisemitic",
"Non-Antisemitic"]

df = df.loc[df['Category'] .isin(list)]
df_platform=df.groupby(['Category', 'Platform'])Instagram.count().unstack('Category').fillna(0).astype(int)


df_platform.plot(kind='barh', stacked=True,mark_right = True )
#איפה שכתובה הרשת נתון לשינוי


KeyError: 'Columns not found: False, True'

In [25]:
#נתונים על הסרת תכנים לפי רשתות
df["Removed 1"]=df["Removed 1"].astype("str") 

def convert_to_category_remove_1(symbol):
    for letter in symbol:
        if letter == "X":
            return "Never"
        elif letter == "V":
            return "Second appeal"
        else:
            return "First appeal"

df["Post status"] = df["Removed 1"].apply(convert_to_category_remove_1)

status = df["Post status"]


#Cheking the percentage of posts removing in general view

status = df["Post status"]

#df.groupby("Category").agg({status:"mean"})

category = df["Category"]


general_removal_view = df.groupby[Instagram].status.value_counts() #get general info about the removal process of platforms

general_removal_view.plot(kind = "pie" , autopct='%1.0f%%', title = "How many posts removed in 1st / 2nd time, or never?", textprops={'color':"w"}) 





TypeError: 'method' object is not subscriptable

In [ ]:
#תכנים שלא הוסרו למרות שקוטלגו כאנטישמיים לפי רשת
df.groupby(['Category', 'Post status'])['Post status'].count().unstack('Post status').fillna(0).astype(int)
